In [47]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd 

# Module relatif à Gurobi
from gurobipy import *

# Module csv
import csv

## Importation des données

In [50]:
# Instance_file : str
Instance_file = "InstanceBordeauxV1.xlsx"

# Chargement des donnees
Employees = pd.read_excel(Instance_file, sheet_name= 'Employees')
Tasks = pd.read_excel(Instance_file, sheet_name= 'Tasks')
Employees_Unavailabilities = pd.read_excel(Instance_file, sheet_name= 'Employees Unavailabilities')
Tasks_Unavailabilities = pd.read_excel(Instance_file, sheet_name= 'Tasks Unavailabilities')

print(Employees)

  EmployeeName   Latitude  Longitude     Skill  Level WorkingStartTime  \
0     Valentin  45.151218  -0.822093  Oenology      2           7:00am   
1        Ambre  45.199575  -0.822093  Oenology      1           8:00am   

  WorkingEndTime  
0         5:00pm  
1         6:00pm  


## Création des dictionnaires de paramètres

Listes des dictionnaires de paramètres à créer:
- LevelEmployee[i] qui à un employé i lui associe son niveau de compétence
- BeginningEmployee[i] qui à un employé i lui associe son début de journée de travail
- EndEmployee[i] qui à un employé i lui associe sa fin de journée de travail
- LevelTask[j] qui à une tâche j lui associe son niveau de difficulté
- BeginningTask[j] qui à une tâche j lui aussi le début du créneau durant lequel on peut l'effectuer
- EndTask[j] qui à une tâche j lui aussi la fin du créneau durant lequel on peut l'effectuer
- DurationTask[j] qui à une tâche j lui associe sa durée
- TasksUnavailabilities[j]
- DistDict[i][j] dictionnaire contenant les distances entre toutes les tâches et les domiciles des employées
- TimeDict[i][j] dictionnaire contenant les temps de trajet associés
- Unavailability[l]: recense les indisponibilités sous forme (employe, debutindispo, duree)

In [63]:
#Paramètres
N = len(Employees["EmployeeName"])
K = len(Tasks["TaskId"])
L = len(Tasks_Unavailabilities["TaskId"])



LevelEmployee = dict()

for i in range (len(Employees["EmployeeName"])):
    a = Employees["EmployeeName"][i]
    b = Employees["Level"][i]
    LevelEmployee[i+1] = b

#print(LevelEmployee)

BeginningEmployee = dict()

for i in range (len(Employees["EmployeeName"])):
    a = Employees["EmployeeName"][i]
    b = Employees["WorkingStartTime"][i][:1]
    bg = b[:2]   #prend les deux premiers caractères
    bd = b[-3:]
    if bg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        b = int(b[:1]) + int(bd[:2]) *100/60
        
    if bg[-1:] != ":":
        b = int(b[:2]) + int(bd[-2:]) *100/60

    if Employees["WorkingStartTime"][i][-2:] == 'am':
        BeginningEmployee[a] = b
    if Employees["WorkingStartTime"][i][2:] == "pm":
        BeginningEmployee[a] = b + 12

#print(BeginningEmployee)

EndEmployee = dict()

for i in range (len(Employees["EmployeeName"])):
    a = Employees["EmployeeName"][i]
    b = Employees["WorkingEndTime"][i][:1]
    bg = b[:2]   #prend les deux premiers caractères
    bd = b[-3:]
    if bg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        b = int(b[:1]) + int(bd[:2]) *100/60
        
    if bg[-1:] != ":":
        b = int(b[:2]) + int(bd[-2:]) *100/60

    if Employees["WorkingEndTime"][i][-2:] == 'am':
        EndEmployee[a] = b
    if Employees["WorkingEndTime"][i][2:] == "pm":
        EndEmployee[a] = b + 12

#print(EndEmployee)

LevelTask = dict()

for i in range (len(Tasks["TaskId"])):
    a = i+1
    b = Tasks["Level"][i]
    LevelTask[a] = b

#print(LevelTask)

TasksUnavailabilities = dict()

for i in range (len(Tasks_Unavailabilities["TaskId"])):
    a = Tasks_Unavailabilities["TaskId"][i] 
    b = Tasks_Unavailabilities["Start"][i][:5]
    bg = b[:2]   #prend les deux premiers caractères
    bd = b[-3:]
    if bg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        b = int(b[:1]) + int(bd[:2]) *100/60
        
    if bg[-1:] != ":":
        b = int(b[:2]) + int(bd[-2:]) *100/60
        
    c = Tasks_Unavailabilities["End"][i][:5]
    cg = c[:2]   #prend les deux premiers caractères
    cd = c[-3:]
    if cg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        c = int(c[:1]) + int(cd[:2]) *100/60
        
    if cg[-1:] != ":":
        c = int(c[:2]) + int(cd[-2:]) *100/60

    if Tasks_Unavailabilities["Start"][i][-2:] == 'pm' and b!=12:
        b = b + 12
    if Tasks_Unavailabilities["End"][i][2:] == "pm" and c!=12:
        c = c + 12
    TasksUnavailabilities[a] = [b,c]

#print(TasksUnavailabilities)

Unavailability = dict()

for i in range (len(Employees_Unavailabilities["EmployeeName"])):
    a = Employees_Unavailabilities["EmployeeName"][i] 
    b = Employees_Unavailabilities["Start"][i][:5]
    bg = b[:2]   #prend les deux premiers caractères
    bd = b[-3:]
    if bg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        b = int(b[:1]) + int(bd[:2]) *100/60
        
    if bg[-1:] != ":":
        b = int(b[:2]) + int(bd[-2:]) *100/60
        
    c = Employees_Unavailabilities["End"][i][:5]
    cg = c[:2]   #prend les deux premiers caractères
    cd = c[-3:]
    if cg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        c = int(c[:1]) + int(cd[:2]) *100/60
        
    if cg[-1:] != ":":
        c = int(c[:2]) + int(cd[-2:]) *100/60
    if Employees_Unavailabilities["Start"][i][-2:] == 'pm'and b!=12:
        b = int(b) + 12
    if Employees_Unavailabilities["End"][i][2:] == "pm" and c!=12:
        c = int(c) + 12 - b
    Unavailability[i] = [a,b,c]

#print(Unavailability)


BeginningTask = dict()

for i in range (K+L+2*N): 
    if i < L:
        a = Employees_Unavailabilities["EmployeeName"][i]
        b = Employees_Unavailabilities["Start"][i][:5]
        c = Employees_Unavailabilities["Start"][i]
    if i >= L and i < L+K:
        a = Tasks["TaskId"][i-L]
        b = Tasks["OpeningTime"][i-L][:5]
        c = Tasks["OpeningTime"][i-L]
    if i >= L+K and i < L+K+N:
        a = Employees["EmployeeName"][i-L-K]
        b = Employees["WorkingStartTime"][i-L-K][:5]
        c = Employees["WorkingStartTime"][i-L-K]
    if i >= L+K+N:
        a = Employees["EmployeeName"][i-L-K-N]
        b = Employees["WorkingEndTime"][i-L-K-N][:5]
        c = Employees["WorkingEndTime"][i-L-K-N]
    bg = b[:2]   #prend les deux premiers caractères
    bd = b[-3:]
    
    if bg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        b = int(b[:1]) + int(bd[:2]) *100/60
        
    if bg[-1:] != ":":
        b = int(b[:2]) + int(bd[-2:]) *100/60
        
    if c[-2:] == 'am':
        #BeginningTask[a] = b
        BeginningTask[i+1] = b
    if c[-2:] == "pm":
        #BeginningTask[a] = b + 12
        BeginningTask[i+1] = b + 12

 

#print(BeginningTask)


EndTask = dict()

for i in range (K+L+2*N): 
    if i < L:
        a = Employees_Unavailabilities["EmployeeName"][i]
        b = Employees_Unavailabilities["End"][i][:5]
        c = Employees_Unavailabilities["End"][i]
    if i >= L and i < L+K:
        a = Tasks["TaskId"][i-L]
        b = Tasks["ClosingTime"][i-L][:5]
        c = Tasks["ClosingTime"][i-L]
    if i >= L+K and i < L+K+N:
        a = Employees["EmployeeName"][i-L-K]
        b = Employees["WorkingStartTime"][i-L-K][:5]
        c = Employees["WorkingStartTime"][i-L-K]
    if i >= L+K+N:
        a = Employees["EmployeeName"][i-L-K-N]
        b = Employees["WorkingEndTime"][i-L-K-N][:5]
        c = Employees["WorkingEndTime"][i-L-K-N]
    bg = b[:2]   #prend les deux premiers caractères
    bd = b[-3:]
    
    if bg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        b = int(b[:1]) + int(bd[:2]) *100/60
        
    if bg[-1:] != ":":
        b = int(b[:2]) + int(bd[-2:]) *100/60
        
    if c[-2:] == 'am':
        #EndTask[a] = b
        EndTask[i+1] = b
    if c[-2:] == "pm":
        #EndTask[a] = b + 12
        EndTask[i+1] = b + 12

#print(EndTask)


DurationTask = dict()

for i in range (K):
    a = L + i + 1
    b = Tasks["TaskDuration"][i] / 60
    DurationTask[a] = b
for i in range (L):
    a = i + 1
    b = Unavailability[i][2] / 60
    DurationTask[a] = b
for i in range (N):
    a = L + K + i + 1
    b = 0
    DurationTask[a] = b
    DurationTask[a+N] = b

#print(DurationTask)

Position = dict()   # contient les positions des tâches indispo

for i in range (L+K+2*N):
    if i < L:
        L_i = Tasks_Unavailabilities["Longitude"][i]
        l_i = Tasks_Unavailabilities["Latitude"][i]
    if i >= L and i < L+K:
        L_i = Tasks["Longitude"][i-L]
        l_i = Tasks["Latitude"][i-L]
    if i >= L+K and i < L+K+N:
        L_i = Employees["Longitude"][i-L-K]
        l_i = Employees["Latitude"][i-L-K]
    if i >= L+K+N :
        L_i = Employees["Longitude"][i-L-K-N]
        l_i = Employees["Latitude"][i-L-K-N]
    Position[i+1] = [l_i, L_i]

        


##Création du dictionnaire des distances
#DistDic: associe à (i,j) la distance entre les deux tâches

DistDict = dict()
R = 6371  #rayon de la Terre

for i in range (1,L+K+2*N+1):               ## ajout des distances entre tasks
    for k in range (1,L+K+2*N+1):
            L_i = Position[i][0]
            l_i = Position[i][1]
            L_k = Position[k][0]
            l_k = Position[k][1]
            A = np.sin((l_i-l_k)*np.pi/(2*180))**2
            B = (np.sin((L_i-L_k)*np.pi/(2*180))**2)*np.cos(l_i*np.pi/180)*np.cos(l_k*np.pi/180)
            d = 2*R*np.sqrt(A+B)
            DistDict[(i,k)] = d


print(DistDict)

##Création dictionnaire des temps de trajet

# TimeDict : associe à un trajet (i,j) le temps de trajet en h


TimeDict = dict()

for element in DistDict.items():
    t = element[1] / 50
    TimeDict[element[0]] = t

#print(TimeDict)



{(1, 1): 0.0, (1, 2): 55.881241192073894, (1, 3): 79.98515242298852, (1, 4): 93.44559398440298, (1, 5): 66.72223630091695, (1, 6): 85.81526900220813, (1, 7): 118.02195632047493, (1, 8): 75.08452637125717, (1, 9): 106.82857904964196, (1, 10): 79.60531046871382, (1, 11): 86.58161111071013, (1, 12): 90.75400519133962, (1, 13): 86.58161111071013, (1, 14): 90.75400519133962, (2, 1): 55.881241192073894, (2, 2): 0.0, (2, 3): 24.104412560139636, (2, 4): 37.70494812210075, (2, 5): 10.852350067908482, (2, 6): 29.99438570218607, (2, 7): 62.23849318666195, (2, 8): 22.93586116195657, (2, 9): 51.361816377472024, (2, 10): 25.359991739004546, (2, 11): 31.310619280070686, (2, 12): 35.05589063463733, (2, 13): 31.310619280070686, (2, 14): 35.05589063463733, (3, 1): 79.98515242298852, (3, 2): 24.104412560139636, (3, 3): 0.0, (3, 4): 14.042626280784043, (3, 5): 13.272246504818561, (3, 6): 6.296951710297473, (3, 7): 38.24909813732715, (3, 8): 15.684842892925245, (3, 9): 27.914836235628954, (3, 10): 10.61875